# Imports do spark sql

In [3]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Imports do spark ML

In [4]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# Imports do Plotly para visualização em gráficos

In [5]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# Configuração básica para utilização do spark

In [6]:
conf = SparkConf().setMaster("local[10]").setAppName("regressao_linear_multipla").set("spark.executor.memory", "14g")
spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()

# Criação do schema do arquivo que será carregado para  o dataframe spark

In [7]:
file_schema = StructType([ StructField("semanas", IntegerType(), True),
                           StructField("clientes", IntegerType(), True),
                           StructField("vendas", DoubleType(), True)])

# Leitura do de arquivo csv para dataframe spark

In [9]:
dataframe = spark.read.format("csv") \
               .option("header", "true") \
               .schema(file_schema) \
               .load('data.csv')

# Visualização de 10 registros do dataframe

In [11]:
dataframe.limit(10).toPandas()

,semanas,clientes,vendas
0,1,907,11.20
1,2,926,11.05
2,3,506,6.84
3,4,741,9.21
4,5,789,9.42
5,6,889,10.08
6,7,874,9.45
7,8,510,6.73
8,9,529,7.24
9,10,420,6.12


# Vetorização das variáveis que serão utilizadas na regressão

In [53]:
# MULTIPLA
feature_assembler = VectorAssembler(inputCols=['semanas','clientes'],outputCol='independent_features')

# Transformação do dataframe para criação da coluna com variáveis vetorizadas

In [15]:
output = feature_assembler.transform(dataframe)
output.limit(10).toPandas()

,semanas,clientes,vendas,independent_features
0,1,907,11.20,"[1.0, 907.0]"
1,2,926,11.05,"[2.0, 926.0]"
2,3,506,6.84,"[3.0, 506.0]"
3,4,741,9.21,"[4.0, 741.0]"
4,5,789,9.42,"[5.0, 789.0]"
5,6,889,10.08,"[6.0, 889.0]"
6,7,874,9.45,"[7.0, 874.0]"
7,8,510,6.73,"[8.0, 510.0]"
8,9,529,7.24,"[9.0, 529.0]"
9,10,420,6.12,"[10.0, 420.0]"


# Separação do dataframe em 75% treinamento e 25% teste

In [46]:
train_data,test_data = output.randomSplit([0.75,0.25])

# Criação do modelo de regressão linear a partir dos dados de trainamento

In [47]:
regressor = LinearRegression(featuresCol='independent_features',labelCol='vendas')
model = regressor.fit(train_data)

# Visualização dos coeficientes e intercept

In [48]:
print("Coefficients: %s" % str(model.coefficients))
print("Intercept: %s" % str(model.intercept))

Coefficients: [-0.026692043582193934,0.008844048591658655]
Intercept: 2.69082501125


# Visualização do número de iterações

In [49]:
training_summary = model.summary
print("numIterations: %d" % training_summary.totalIterations)
print("objectiveHistory: %s" % str(training_summary.objectiveHistory))

numIterations: 1
objectiveHistory: [0.0]


# Visualização de residuo

In [50]:
training_summary.residuals.limit(10).toPandas()

,residuals
0,0.514315
1,0.222970
2,-0.245837
3,0.072503
4,-0.115319
5,-0.257753
6,0.110902
7,-0.018405
8,-0.772322
9,-0.652531


# Visualização do RMSE e do r2 

In [51]:
print("RMSE: %f" % training_summary.rootMeanSquaredError)
print("r2: %f" % training_summary.r2)

RMSE: 0.432641
r2: 0.929283


# Utilização dos dados de teste para verificar resultado de previsão

In [52]:
pred_result = model.evaluate(test_data)
pred_result.predictions.limit(10).toPandas()

,semanas,clientes,vendas,independent_features,prediction
0,6,889,10.08,"[6.0, 889.0]",10.393032
1,7,874,9.45,"[7.0, 874.0]",10.233679
2,13,924,9.46,"[13.0, 924.0]",10.515729
3,16,729,8.95,"[16.0, 729.0]",8.711064
4,17,794,9.33,"[17.0, 794.0]",9.259235
5,19,1010,11.77,"[19.0, 1010.0]",11.116165
